In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

import numpy as np

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sn

pd.set_option('display.max_rows', 300) 
pd.options.display.float_format = '{:40,.4f}'.format 
plt.style.use('ggplot') 
%matplotlib inline

# Read the file

In [ ]:
LONDON = pd.read_csv('LONDON.csv')
LONDONDATA=LONDON.drop(['Unnamed: 0','year'], axis=1)

# Pearson Correlation Analysis

In [ ]:
df = LONDONDATA
plt.rcParams["axes.grid"] = False
f = plt.figure(figsize=(19, 15))
plt.matshow(df.corr(), fignum=f.number)
plt.xticks(range(df.shape[1]), df.columns, fontsize=14, rotation=45)
plt.yticks(range(df.shape[1]), df.columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=15) 
plt.savefig('./MATRIX.png')
plt.title('Pearson Correlation Matrix', fontsize=16)
RELAT=df.corr()
RELAT.to_csv('./CORELATION.csv')

# Regression Analysis

In [ ]:
model1 = sm.formula.ols('cases_total ~ pop_total + male_to_female + total_budget + clean_air + clean_environment + health_training + school_awareness + media_awareness+sub_counselling', LONDONLOC).fit()
model1.summary()

# VIF

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from statsmodels.tools.tools import add_constant

def drop_column_using_vif_(df, thresh=5):
    while True:
        df_with_const = add_constant(df)

        vif_df = pd.Series([variance_inflation_factor(df_with_const.values, i) 
               for i in range(df_with_const.shape[1])], name= "VIF",
              index=df_with_const.columns).to_frame()

        vif_df = vif_df.drop('const')
        if vif_df.VIF.max() > thresh:
            index_to_drop = vif_df.index[vif_df.VIF == vif_df.VIF.max()].tolist()[0]
            print('Dropping: {}'.format(index_to_drop))
            df = df.drop(columns = index_to_drop)
        else:
            break

    return df
model_new = drop_column_using_vif_(LONDONLOC.drop('cases_total', axis=1))

# Do the Regression Analysis Again

In [ ]:
model_afterVIF = sm.formula.ols('cases_total ~ pop_total + male_to_female + clean_air + clean_environment', LONDONLOC).fit()
model_afterVIF.sumary()

In [ ]:
plt.scatter(LONDONmodel1.fittedvalues, LONDONmodel.resid)
plt.hlines(0,0,1300,color="black",linestyles="--")
plt.xlabel('cases of childhood obesity')
plt.ylabel('Residual')
plt.savefig('./Residual_y.png')
################################
fig = plt.figure(figsize=(12,8))
fig = sm.graphics.plot_regress_exog(LONDONmodel1, 'clean_environment', fig=fig)
plt.savefig('./environment.png')